# <font color='blue'>Uni-Facef - PySpark Parte 3 </font>

Este notebook simula um response de uma API e que recebe uma mensagem no padrão Json. Com isso, esse dataset receberá o tratamento necessário para para fazer o "parsing" da mensagem e a normalização estruturada da informação 

In [ ]:
# coding: utf-8
import pyspark.sql.functions as sf
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName('Parsing Json') \
    .getOrCreate()

In [ ]:
response = {"body": [
    {"id": "1000",
     "nome": "André",
     "endereco": {"logradouro": "Rua Joaquim Cruz",
                  "numero": "534",
                  "cidade": "Franca"},
     "tags": ["Grupo1", "Grupo2"],
     "dt_ult_atualizacao":"2020-08-09 00:00:00"},
    {"id": "1001",
     "nome": "Felipe",
     "endereco": {"logradouro": "Av Nova Lua",
                  "numero": "1093",
                  "cidade": "São Paulo"},
     "tags": ["Grupo1", "Grupo3"],
     "dt_ult_atualizacao":"2020-08-11 00:00:00"},
    {"id": "1002",
     "nome": "Maria",
     "endereco": {"logradouro": "Rua J",
                  "numero": "10",
                  "cidade": "Araraquara"},
     "tags": ["Grupo3"],
     "dt_ult_atualizacao":"2020-08-13 00:00:00"}
]}

response

In [ ]:
print(type(response))
print(type(response['body']))

In [ ]:
df = spark.createDataFrame(response['body'])
df.show()

In [ ]:
# O DataFrame não reconhece a estrutura no segundo nível do endereço
df.printSchema()

#### Criando o mesmo DataFrame apartir de um RDD usando a função parallelize() e map()

- parallelize() - É uma função em SparkContext e é usado para criar um RDD de uma coleção de lista.

- map() - É o método do RDD da qual recebe uma função como parâmetro e passa por todos os elementos do RDD

In [ ]:
import json

rdd = sc.parallelize(response['body']).map(lambda x: json.dumps(x))
df = spark.read.json(rdd)
df.printSchema()

#### Normalizando as informações de primeiro nível do documento

O campo endereço é um objeto do tipo "struct", onde armazena informações num segundo nível. Porém nesse caso possui uma relação 1:1 com os campos do primeiro. Para acessar a informação: "campo_1_nivel.campo_2_nivel"

``` 
{'id': '1000',
 'nome': 'André',
 'endereco': {'logradouro': 'Rua Joaquim Cruz',
              'numero': '534',
              'cidade': 'Franca'},
 'tags': ['Grupo1', 'Grupo2'],
 'dt_ult_atualizacao': '2020-08-08 00:00:00'}
```

In [ ]:
cliente = df.select(
    sf.col("id").alias("id_cliente"),
    sf.col("nome").alias("nome_cliente"),
    sf.col("endereco.logradouro").alias("end_logradouro"),
    sf.col("endereco.numero").alias("numero"),
    sf.col("endereco.cidade").alias("cidade"),
    sf.col("dt_ult_atualizacao").alias("dt_ult_atualizacao")) \
    .withColumn("dt_ult_atualizacao", sf.col("dt_ult_atualizacao").cast('timestamp'))

cliente.show()

#### Normalizando as informações de tags que possui uma relação 1:N

```
'tags': ['Grupo1', 'Grupo2']
```
Nesse caso é preciso normalizar estruturando as informações de "tag" em um novo dataset relacionável com o primeiro através da chave "id_cliente". usando a funções "explode()"

In [ ]:
cliente_tag = df.select(
    sf.col("id").alias("id_cliente"),
    sf.explode("tags").alias("tag"))

cliente_tag.show()

#### Filtra os clientes do "Grupo3" 

- Faz o filtro no dataframe "cliente_tag" através do método "filter()"
- Faz join com o dataframe "cliente" através do método "join()"
- Renomea a coluna "tag" usando o método "withColumnRenamed()"
- Cria um novo campo chamado "endereco_completo" usando o método "withColumn"
    - Para concatenação é utilizada a função "concat" 
    - Para concatenar valores "fixos" é utilizada a função "lit()"

In [ ]:
cli_grupo3 = cliente_tag \
    .filter(cliente_tag.tag == "Grupo3") \
    .join(cliente, 'id_cliente', 'inner') \
    .withColumnRenamed("tag", "tag_cliente") \
    .withColumn(
        "endereco_completo", 
        sf.concat("end_logradouro", sf.lit(", "), "numero"))


cli_grupo3.show()

#### Gravando um dataset com mode append e partitionBy

In [ ]:
from datetime import datetime

cliente = cliente \
    .withColumn("datalog", sf.lit(str(datetime.utcnow().date())))

cliente.show(5)

In [ ]:
# Escreve em formato parquet
cliente.write.parquet(
    "cliente", 
    mode="append", 
    partitionBy=["datalog"])

In [ ]:
!pwd

In [ ]:
!ls -l /home/al_morais/Documentos/UniFacef/notebooks/cliente

In [ ]:
!ls -l /home/al_morais/Documentos/UniFacef/notebooks/cliente/datalog=2020-09-27

#### Gravar o dataset em CSV
Vamos gravar o dataset em CSV simulando um Job Sqoop fazendo uma ingestão incremental de uma tabela de um banco relacional 

In [ ]:
cliente.printSchema()

In [ ]:
# Escreve em formato CSV
cliente.repartition(1).write \
    .option("delimiter", "|") \
    .csv("cliente_csv", mode="append")

In [ ]:
!ls -l /home/al_morais/Documentos/UniFacef/notebooks/cliente_csv/

### FIM
###### Documentação: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html